In [ ]:
#| default_exp statistics

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#| export
import scipy.sparse as sp, re, xclib.data.data_utils as du, numpy as np, pandas as pd, os
from IPython.display import display
from torch.utils.data import Dataset
from termcolor import colored, COLORS

from sugar.core import *
from xcai.data import *

## `UpdatedDataset`

In [ ]:
#| export
METADATA_CODE = {'category': 'cat', 'see_also': 'sal', 'hyper_link': 'hlk', 'videos': 'vid', 'images': 'img', 'entity_gpt': 'ent', 'entity_llama': 'ent', 
                 'entity': 'ent', 'canonical': 'can', 'entity_canonical_category': 'ecc', 'entity_canonical': 'enc'}

In [ ]:
#| export
class UpdatedDataset:

    @staticmethod
    def load_data_info(data_dir, type, suffix=''):
        if len(suffix): suffix = f'.{suffix}'
        ids, txt = load_raw_file(f'{data_dir}/raw_data/{type}{suffix}.raw.csv')
        return {'identifier':ids, 'input_text':txt}

    @staticmethod
    def load_lbl_info(data_dir, x_prefix, y_prefix):
        ids, txt = load_raw_file(f'{data_dir}/raw_data/label.{x_prefix}-{y_prefix}.raw.csv')
        return {'identifier':ids, 'input_text':txt}

    @staticmethod
    def load_metadata_info(data_dir, metadata_type, x_prefix, y_prefix, z_prefix):
        ids, txt = load_raw_file(f'{data_dir}/raw_data/{metadata_type}.{x_prefix}-{y_prefix}-{z_prefix}.raw.csv')
        return {'identifier':ids, 'input_text':txt}

    @staticmethod
    def get_trn_tst_info(data_dir, suffix=''):
        trn_info = UpdatedDataset.load_data_info(data_dir, 'train', suffix)
        tst_info = UpdatedDataset.load_data_info(data_dir, 'test', suffix)
        return trn_info, tst_info

    @staticmethod
    def load_main_matrix(data_dir, x_prefix, y_prefix, type):
        if os.path.exists(f'{data_dir}/{type}_X_Y_{x_prefix}-{y_prefix}.npz'):
            mat = sp.load_npz(f'{data_dir}/{type}_X_Y_{x_prefix}-{y_prefix}.npz')
        else:
            mat = du.read_sparse_file(f'{data_dir}/{type}_X_Y_{x_prefix}-{y_prefix}.txt')
        return mat

    @staticmethod
    def get_labels(data_dir, x_prefix, y_prefix):
        trn_mat = UpdatedDataset.load_main_matrix(data_dir, x_prefix, y_prefix, 'trn')
        tst_mat = UpdatedDataset.load_main_matrix(data_dir, x_prefix, y_prefix, 'tst')
        
        lbl_info = UpdatedDataset.load_lbl_info(data_dir, x_prefix=x_prefix, y_prefix=y_prefix)
        
        return trn_mat, tst_mat, lbl_info

    @staticmethod
    def load_metadata_matrix(data_dir, x_prefix, y_prefix, z_prefix, main_type, metadata_type):
        if os.path.exists(f'{data_dir}/{metadata_type}_{main_type}_X_Y_{x_prefix}-{y_prefix}-{z_prefix}.npz'):
            mat = sp.load_npz(f'{data_dir}/{metadata_type}_{main_type}_X_Y_{x_prefix}-{y_prefix}-{z_prefix}.npz')
        else:
            mat = du.read_sparse_file(f'{data_dir}/{metadata_type}_{main_type}_X_Y_{x_prefix}-{y_prefix}-{z_prefix}.txt')
        return mat

    @staticmethod
    def get_metadata(data_dir, metadata_type, x_prefix, y_prefix, z_prefix):
        trn_mat = UpdatedDataset.load_metadata_matrix(data_dir, x_prefix, y_prefix, z_prefix, 'trn', metadata_type)
        tst_mat = UpdatedDataset.load_metadata_matrix(data_dir, x_prefix, y_prefix, z_prefix, 'tst', metadata_type)
        lbl_mat = UpdatedDataset.load_metadata_matrix(data_dir, x_prefix, y_prefix, z_prefix, 'lbl', metadata_type)
        
        meta_info = UpdatedDataset.load_metadata_info(data_dir, metadata_type, x_prefix, y_prefix, z_prefix)
        
        return trn_mat, tst_mat, lbl_mat, meta_info

    @staticmethod
    def load_datasets(data_dir, metadata_type, x_prefix, y_prefix, z_prefix):
        trn_info, tst_info = UpdatedDataset.get_trn_tst_info(data_dir, x_prefix)
        trn_mat, tst_mat, lbl_info = UpdatedDataset.get_labels(data_dir, x_prefix, y_prefix)
    
        main_trn_dset = MainXCDataset(trn_info, trn_mat, lbl_info)
        main_tst_dset = MainXCDataset(tst_info, tst_mat, lbl_info)
    
        trn_meta_mat, tst_meta_mat, lbl_meta_mat, meta_info = UpdatedDataset.get_metadata(data_dir, metadata_type, x_prefix, y_prefix, z_prefix)
    
        trn_meta_dset = MetaXCDataset(METADATA_CODE[metadata_type], trn_meta_mat, lbl_meta_mat, meta_info)
        tst_meta_dset = MetaXCDataset(METADATA_CODE[metadata_type], tst_meta_mat, lbl_meta_mat, meta_info)
    
        trn_dset = XCDataset(main_trn_dset, cat_meta=trn_meta_dset)
        tst_dset = XCDataset(main_tst_dset, cat_meta=tst_meta_dset)
    
        return trn_dset, tst_dset
        

## `Dataset`

In [ ]:
#| export
class Dataset:

    @staticmethod
    def load_data_info(data_dir, type, suffix="", encoding='utf-8'):
        if len(suffix): suffix = f".{suffix}"
        fname = f'{data_dir}/raw_data/{type}{suffix}.raw'
        ids, txt = load_raw_file(fname+'.csv') if os.path.exists(fname+'.csv') else load_raw_file(fname+'.txt', encoding=encoding)
        return {'identifier':ids, 'input_text':txt}

    @staticmethod
    def load_lbl_info(data_dir, type, suffix="", encoding='utf-8', lbl_suffix=''):
        if len(suffix): suffix = f".{suffix}-{suffix}"
        elif len(lbl_suffix): suffix = f".{lbl_suffix}"
            
        fname = f'{data_dir}/raw_data/{type}{suffix}.raw'
        ids, txt = load_raw_file(fname+'.csv') if os.path.exists(fname+'.csv') else load_raw_file(fname+'.txt', encoding=encoding)
        return {'identifier':ids, 'input_text':txt}

    @staticmethod
    def load_metadata_info(data_dir, metadata_type, suffix="", encoding='utf-8'):
        if len(suffix): suffix = f".{suffix}-{suffix}-{suffix}"
        fname = f'{data_dir}/raw_data/{metadata_type}{suffix}.raw'
        ids, txt = load_raw_file(fname+'.csv') if os.path.exists(fname+'.csv') else load_raw_file(fname+'.txt', encoding=encoding)
        return {'identifier':ids, 'input_text':txt}

    @staticmethod
    def get_trn_tst_info(data_dir, suffix="", encoding='utf-8'):
        trn_info = Dataset.load_data_info(data_dir, 'train', suffix, encoding)
        tst_info = Dataset.load_data_info(data_dir, 'test', suffix, encoding)
        return trn_info, tst_info

    @staticmethod
    def get_labels(data_dir, suffix="", encoding='utf-8', lbl_suffix=''):
        if len(lbl_suffix): suffix = f'_{lbl_suffix}'
        trn_mat = du.read_sparse_file(f'{data_dir}/trn_X_Y{suffix}.txt') if os.path.exists(f'{data_dir}/trn_X_Y{suffix}.txt') else sp.load_npz(f'{data_dir}/trn_X_Y{suffix}.npz')
        tst_mat = du.read_sparse_file(f'{data_dir}/tst_X_Y{suffix}.txt') if os.path.exists(f'{data_dir}/tst_X_Y{suffix}.txt') else sp.load_npz(f'{data_dir}/tst_X_Y{suffix}.npz')
            
        lbl_info = Dataset.load_lbl_info(data_dir, 'label', suffix, encoding, lbl_suffix='')
        
        return trn_mat, tst_mat, lbl_info

    @staticmethod
    def get_metadata(data_dir, metadata_type, suffix="", encoding='utf-8'):
        trn_mat = du.read_sparse_file(f'{data_dir}/{metadata_type}_trn_X_Y.txt') if os.path.exists(f'{data_dir}/{metadata_type}_trn_X_Y.txt') else sp.load_npz(f'{data_dir}/{metadata_type}_trn_X_Y.npz')
        tst_mat = du.read_sparse_file(f'{data_dir}/{metadata_type}_tst_X_Y.txt') if os.path.exists(f'{data_dir}/{metadata_type}_tst_X_Y.txt') else sp.load_npz(f'{data_dir}/{metadata_type}_tst_X_Y.npz')
        lbl_mat = du.read_sparse_file(f'{data_dir}/{metadata_type}_lbl_X_Y.txt') if os.path.exists(f'{data_dir}/{metadata_type}_lbl_X_Y.txt') else sp.load_npz(f'{data_dir}/{metadata_type}_lbl_X_Y.npz')
        
        meta_info = Dataset.load_metadata_info(data_dir, metadata_type, suffix, encoding)
        
        return trn_mat, tst_mat, lbl_mat, meta_info

    @staticmethod
    def load_datasets(data_dir, metadata_type, suffix="", encoding='utf-8', main_mat_suffix=''):
        trn_info, tst_info = Dataset.get_trn_tst_info(data_dir, suffix, encoding)
        trn_mat, tst_mat, lbl_info = Dataset.get_labels(data_dir, suffix, encoding, main_mat_suffix)
    
        main_trn_dset = MainXCDataset(trn_info, trn_mat, lbl_info)
        main_tst_dset = MainXCDataset(tst_info, tst_mat, lbl_info)
    
        trn_meta_mat, tst_meta_mat, lbl_meta_mat, meta_info = Dataset.get_metadata(data_dir, metadata_type, suffix, encoding)
    
        trn_meta_dset = MetaXCDataset(METADATA_CODE[metadata_type], trn_meta_mat, lbl_meta_mat, meta_info)
        tst_meta_dset = MetaXCDataset(METADATA_CODE[metadata_type], tst_meta_mat, lbl_meta_mat, meta_info)
    
        trn_dset = XCDataset(main_trn_dset, cat_meta=trn_meta_dset)
        tst_dset = XCDataset(main_tst_dset, cat_meta=tst_meta_dset)
    
        return trn_dset, tst_dset
        

## Compute statistics

In [ ]:
#| export
CODE_METADATA = {'cat':'Category', 'sal':'Seealso', 'hlk':'Hyperlink', 'vid':'Videos', 'img':'Images', 
                 'ent':'Entity', 'can':'Canonical', 'ecc':'Entity Canonical Category', 'enc':'Entity Canonical'}

In [ ]:
#| export
def matrix_stats(mat):
    n_dat = mat.shape[0]
    n_lbl = mat.shape[1]

    num_dat_lbl = mat.getnnz(axis=0)
    num_lbl_dat = mat.getnnz(axis=1)

    avg_dat_lbl = num_dat_lbl.mean()
    avg_lbl_dat = num_lbl_dat.mean()

    max_dat_lbl = num_dat_lbl.max()
    max_lbl_dat = num_lbl_dat.max()

    zro_dat_lbl = np.sum(num_dat_lbl == 0)
    zro_lbl_dat = np.sum(num_lbl_dat == 0)

    stats_dict = {
        f'# Entries' : n_dat,
        f'# Features': n_lbl,
        f'Avg. Entries per feature' : avg_dat_lbl,
        f'Avg. Feature per entry'   : avg_lbl_dat,
        f'Max. Entries per feature' : max_dat_lbl,
        f'Max. Feature per entry'   : max_lbl_dat,
        f'# Features without entry' : zro_dat_lbl,
        f'# Entries without feature': zro_lbl_dat,
    }
    return stats_dict
    

In [ ]:
#| export
def main_dset_stats(dset):
    return matrix_stats(dset.data_lbl)
    

In [ ]:
#| export
def meta_dset_stats(dset):
    dat_stats = matrix_stats(dset.data_meta)
    dat_stats['Dataset'] = 'Query'
    
    lbl_stats = matrix_stats(dset.lbl_meta)
    lbl_stats['Dataset'] = 'Label'

    return [dat_stats, lbl_stats]
    

In [ ]:
#| export
def dset_stats(dset):
    stats = []
    
    main_stats = main_dset_stats(dset.data)
    main_stats['Dataset'] = 'Main'
    stats.append(main_stats)

    for o in dset.meta.values():
        meta_stats = meta_dset_stats(o)
        for s in meta_stats: s['Dataset'] = f'{s["Dataset"]} {CODE_METADATA[o.prefix]} Metadata'
        stats.extend(meta_stats)

    return stats
    

In [ ]:
#| export
def trn_tst_stats(trn_dset, tst_dset):
    trn_stats = dset_stats(trn_dset)
    for o in trn_stats: o['Split'] = 'Train'

    tst_stats = dset_stats(tst_dset)
    for o in tst_stats: o['Split'] = 'Test'

    stats = trn_stats + tst_stats
    return stats
    

In [ ]:
#| export
def print_stats(stats):
    df = pd.DataFrame(stats).set_index(['Split', 'Dataset'])
    with pd.option_context('display.precision', 2):
        display(df)
    

In [ ]:
#| export
def print_dset_stats(trn_dset, tst_dset):
    stats = trn_tst_stats(trn_dset, tst_dset)
    print_stats(stats)
    

## Text

In [ ]:
#| export
class TextDataset(Dataset):
    
    def __init__(self, dset, pattern='.*_text$'):
        self.dset, self.pattern = dset, pattern
        colors = list(COLORS.keys())
        self.colors = [colors[i] for i in np.random.permutation(len(colors))]
    
    def __getitem__(self, idx):
        o = self.dset[idx]
        return {k:v for k,v in o.items() if re.match(self.pattern, k)}

    def show(self, idxs):
        for idx in idxs:
            for i,(k,v) in enumerate(self[idx].items()):
                key = colored(k, self.colors[i], attrs=["reverse", "blink"])
                value = colored(f': {v}', self.colors[i])
                print(key, value)
            print()

    def get_head_data(self, topk=10):
        return np.argsort(self.dset.data.data_lbl.getnnz(axis=1))[:-topk:-1]

    def get_tail_data(self, topk=10):
        num = self.dset.data.data_lbl.getnnz(axis=1)
        idx = np.argsort(num)
        valid = (num > 0)[idx]
        return idx[valid][:topk]
        
    def dump_txt(self, fname, idxs):
        with open(fname, 'w') as file:
            for idx in idxs:
                for i,(k,v) in enumerate(self[idx].items()):
                    file.write(f'{k}: {v}\n')
                file.write('\n')
            
    def dump_csv(self, fname, idxs):
        df = pd.DataFrame([self[idx] for idx in idxs])
        df.to_csv(fname, index=False)

    def dump(self, fname, idxs):
        if fname.endswith('.txt'): 
            self.dump_txt(fname, idxs)
        elif fname.endswith('.csv'): 
            self.dump_csv(fname, idxs)
        else: 
            raise ValueError(f'Invalid file extension: {fname}')
        
    

## Compare dataset

In [ ]:
#| export
class CompareDataset:

    def __init__(self, old_dset, new_dset):
        self.old_dset, self.new_dset = old_dset, new_dset
        colors = list(COLORS.keys())
        self.colors = [colors[i] for i in np.random.permutation(len(colors))]

    def data(self, topk=10, idx_type=''):
        assert self.old_dset.n_data == self.new_dset.n_data, f'Different number of datapoints in the two datasets.'

        old_lbl_dat = self.old_dset.data.data_lbl.getnnz(axis=1)
        new_lbl_dat = self.new_dset.data.data_lbl.getnnz(axis=1)

        if idx_type == 'max': idxs = np.argsort(new_lbl_dat - old_lbl_dat)[:-topk:-1]
        elif idx_type == 'min': idxs = np.argsort(new_lbl_dat - old_lbl_dat)[:topk]
        else: idxs = np.random.permutation(self.old_dset.n_data)[:topk]
        
        for idx in idxs:
            old_item = self.old_dset[idx]
            new_item = self.new_dset[idx]
            
            key = colored("data_input_text", self.colors[0], attrs=["reverse", "blink"])
            value = colored(f': {old_item["data_input_text"]}', self.colors[0])
            print(key, value)

            key = colored("old lbl2data_input_text", self.colors[1], attrs=["reverse", "blink"])
            value = colored(f': {old_item["lbl2data_input_text"]}', self.colors[1])
            print(key, value)

            key = colored("new lbl2data_input_text", self.colors[2], attrs=["reverse", "blink"])
            value = colored(f': {new_item["lbl2data_input_text"]}', self.colors[2])
            print(key, value)
            
            print()

    def metadata(self, metadata_type, data_type='data', topk=10, idx_type=''):
        assert self.old_dset.n_data == self.new_dset.n_data, f'Different number of datapoints in the two datasets.'

        if data_type == 'data':
            pattern = r"^(?!lbl).{3}2data_input_text"
            old_lbl_dat = self.old_dset.meta[metadata_type].data_meta.getnnz(axis=1)
            new_lbl_dat = self.new_dset.meta[metadata_type].data_meta.getnnz(axis=1)
        elif data_type == 'lbl':
            pattern = r"^.{3}2lbl2data_input_text"
            old_lbl_dat = self.old_dset.meta[metadata_type].lbl_meta.getnnz(axis=1)
            new_lbl_dat = self.new_dset.meta[metadata_type].lbl_meta.getnnz(axis=1)

        if idx_type == 'max': idxs = np.argsort(new_lbl_dat - old_lbl_dat)[:-topk:-1]
        elif idx_type == 'min': idxs = np.argsort(new_lbl_dat - old_lbl_dat)[:topk]
        else: idxs = np.random.permutation(self.old_dset.n_data)[:topk]
        
        for idx in idxs:
            old_item = self.old_dset[idx]
            new_item = self.new_dset[idx]

            key = colored("data_input_text", self.colors[0], attrs=["reverse", "blink"])
            value = colored(f': {old_item["data_input_text"]}', self.colors[0])
            print(key, value)

            key = colored("old lbl2data_input_text", self.colors[1], attrs=["reverse", "blink"])
            value = colored(f': {new_item["lbl2data_input_text"]}', self.colors[1])
            print(key, value)

            key = colored("new lbl2data_input_text", self.colors[2], attrs=["reverse", "blink"])
            value = colored(f': {new_item["lbl2data_input_text"]}', self.colors[2])
            print(key, value)

            ctr = 0
            for i,k in enumerate(old_item):
                if re.match(pattern, k):
                    key = colored(f"old {k}", self.colors[2*ctr+3], attrs=["reverse", "blink"])
                    value = colored(f': {old_item[k]}', self.colors[2*ctr+3])
                    print(key, value)
    
                    key = colored(f"new {k}", self.colors[2*ctr+4], attrs=["reverse", "blink"])
                    value = colored(f': {new_item[k]}', self.colors[2*ctr+4])
                    print(key, value)

                    ctr += 1
            
            print()
            

## Save dataset

In [ ]:
#| export
PREFIX_METDATA = {'cat': 'category', 'hlk': 'hyper_link', 'sal': 'see_also'}

In [ ]:
#| export
def save_labels(data_dir, trn_dset, tst_dset):
    os.makedirs(data_dir, exist_ok=True)

    sp.save_npz(f'{data_dir}/trn_X_Y.npz', trn_dset.data.data_lbl)
    sp.save_npz(f'{data_dir}/tst_X_Y.npz', tst_dset.data.data_lbl)

    os.makedirs(f'{data_dir}/raw_data', exist_ok=True)

    save_raw_file(f'{data_dir}/raw_data/train.raw.txt', trn_dset.data.data_info['identifier'], trn_dset.data.data_info['input_text'])
    save_raw_file(f'{data_dir}/raw_data/test.raw.txt', tst_dset.data.data_info['identifier'], tst_dset.data.data_info['input_text'])
    save_raw_file(f'{data_dir}/raw_data/label.raw.txt', trn_dset.data.lbl_info['identifier'], trn_dset.data.lbl_info['input_text'])
    

In [ ]:
#| export
def save_metadata(data_dir, trn_dset, tst_dset):
    metadata_type = None
    
    for metadata in trn_dset.meta.keys():
        metadata_type = PREFIX_METDATA[trn_dset.meta[metadata].prefix]
        
        sp.save_npz(f'{data_dir}/{metadata_type}_trn_X_Y.npz', trn_dset.meta[metadata].data_meta)
        sp.save_npz(f'{data_dir}/{metadata_type}_tst_X_Y.npz', tst_dset.meta[metadata].data_meta)
        sp.save_npz(f'{data_dir}/{metadata_type}_lbl_X_Y.npz', trn_dset.meta[metadata].lbl_meta)
        
        save_raw_file(f'{data_dir}/raw_data/{metadata_type}.raw.txt', trn_dset.meta[metadata].meta_info['identifier'], trn_dset.meta[metadata].meta_info['input_text'])
    

In [ ]:
#| export
def save_dataset(data_dir, trn_dset, tst_dset):
    valid_idx = np.where(trn_dset.data.data_lbl.getnnz(axis=1) > 0)[0]
    trn_dset = trn_dset._getitems(valid_idx)

    valid_idx = np.where(tst_dset.data.data_lbl.getnnz(axis=1) > 0)[0]
    tst_dset = tst_dset._getitems(valid_idx)

    save_labels(data_dir, trn_dset, tst_dset)
    save_metadata(data_dir, trn_dset, tst_dset)

    return trn_dset, tst_dset
    

## Helper

In [ ]:
#| export
def show_updated_dataset(data_dir, metadata_type, x_prefix, y_prefix, z_prefix, idxs, use_trn=True):
    trn_dset, tst_dset = UpdatedDataset.load_datasets(data_dir, metadata_type, x_prefix, y_prefix, z_prefix)
    print_dset_stats(trn_dset, tst_dset)
    
    txt_dset = TextDataset(trn_dset if use_trn else tst_dset)
    txt_dset.show(idxs)
    
    return trn_dset, tst_dset

def show_dataset(data_dir, metadata_type, idxs, encoding='utf-8', use_trn=True):
    trn_dset, tst_dset = Dataset.load_datasets(data_dir, metadata_type, encoding=encoding)
    print_dset_stats(trn_dset, tst_dset)
    
    txt_dset = TextDataset(trn_dset if use_trn else tst_dset)
    txt_dset.show(idxs)
    
    return trn_dset, tst_dset
    

## `__main__`

In [ ]:
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, required=True)
    parser.add_argument('--save_dir', type=str, required=True)
    parser.add_argument('--metadata_key', type=str, required=None)
    parser.add_argument('--x_prefix', type=str, required=True)
    parser.add_argument('--y_prefix', type=str, required=True)
    parser.add_argument('--z_prefix', type=str, required=True)
    return parser.parse_args()
    

In [ ]:
if __name__ == '__main__':
    args = parse_args()
    trn_dset, tst_dset = UpdatedDataset.load_datasets(args.data_dir, args.save_dir, args.metadata_type, 
                                                      args.x_prefix, args.y_prefix, args.z_prefix)
    

In [ ]:
data_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/(mapped)LF-WikiSeeAlsoTitles-320K/'
save_dir = '/home/scai/phd/aiz218323/scratch/datasets/wikipedia/20250123/LF-WikiSeeAlsoTitles-320K/'

metadata_type = 'category'
x_prefix, y_prefix, z_prefix = 'new', 'new', 'new'

In [ ]:
trn_dset, tst_dset = UpdatedDataset.load_datasets(save_dir, metadata_type, x_prefix, y_prefix, z_prefix)

In [ ]:
new_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/20250123-LF-WikiSeeAlsoTitles-320K/'

In [ ]:
new_trn_dset, new_tst_dset = save_dataset(new_dir, trn_dset, tst_dset)

In [ ]:
print_dset_stats(new_trn_dset, new_tst_dset)

# Entries  # Features  \
Split Dataset                                          
Train Main                       1183189     1205595   
      Query Category Metadata    1183189     1155247   
      Label Category Metadata    1205595     1155247   
Test  Main                        340278     1205595   
      Query Category Metadata     340278     1155247   
      Label Category Metadata    1205595     1155247   

                               Avg. Entries per feature  \
Split Dataset                                             
Train Main                                         2.57   
      Query Category Metadata                      5.21   
      Label Category Metadata                      3.75   
Test  Main                                         0.82   
      Query Category Metadata                      1.49   
      Label Category Metadata                      3.75   

                               Avg. Feature per entry  \
Split Dataset                                           
Train Main                                       2.62   
      Query Category Metadata                    5.09   
      Label Category Metadata                    3.60   
Test  Main                                       2.90   
      Query Category Metadata                    5.06   
      Label Category Metadata                    3.60   

                               Max. Entries per feature  \
Split Dataset                                             
Train Main                                         4747   
      Query Category Metadata                     65056   
      Label Category Metadata                     25831   
Test  Main                                         1075   
      Query Category Metadata                     17104   
      Label Category Metadata                     25831   

                               Max. Feature per entry  \
Split Dataset                                           
Train Main                                        987   
      Query Category Metadata                     245   
      Label Category Metadata                     247   
Test  Main                                        150   
      Query Category Metadata                     247   
      Label Category Metadata                     247   

                               # Features without entry  \
Split Dataset                                             
Train Main                                            0   
      Query Category Metadata                    159691   
      Label Category Metadata                    280544   
Test  Main                                       758109   
      Query Category Metadata                    625513   
      Label Category Metadata                    280544   

                               # Entries without feature  
Split Dataset                                             
Train Main                                             0  
      Query Category Metadata                     102390  
      Label Category Metadata                     420595  
Test  Main                                             0  
      Query Category Metadata                      24033  
      Label Category Metadata                     420595

In [ ]:
txt_dset = TextDataset(new_trn_dset)

In [ ]:
os.makedirs(f'{new_dir}/examples', exist_ok=True)

idxs = np.random.permutation(new_trn_dset.n_data)[:1000]
txt_dset.dump(f'{new_dir}/examples/random_train.txt', idxs)

In [ ]:
txt_dset.show([10, 20, 30])

data_input_text : Arithmetic mean
lbl2data_input_text : ['Fréchet mean', 'Generalized mean', 'Summary statistics', 'Standard deviation', 'Standard error of the mean', 'Sample mean and covariance', 'Inequality of arithmetic and geometric means']
cat2data_input_text : ['Means']
cat2lbl2data_input_text : [['Means'], ['Means', 'Inequalities', 'Articles with example Haskell code'], ['Summary statistics'], ['Summary statistics', 'Statistical deviation and dispersion'], [], ['U-statistics', 'Summary statistics', 'Estimation methods', 'Covariance and correlation', 'Matrices'], []]

data_input_text : Annual plant
lbl2data_input_text : ['Ephemeral plant']
cat2data_input_text : ['Annual plants']
cat2lbl2data_input_text : [['Ephemeral plants', 'Plants', 'Flowers']]

data_input_text : Audi
lbl2data_input_text : ['DKW', 'Wanderer (company)', 'Volkswagen Group', 'Horch']
cat2data_input_text : ['Companies based in Ingolstadt', 'Volkswagen Group', 'Re-established companies', 'Sports car manufacturers',

In [ ]:
old_trn_dset, old_tst_dset = UpdatedDataset.load_datasets(save_dir, metadata_type, x_prefix='old', y_prefix='old', z_prefix='old')

/home/scai/phd/aiz218323/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [ ]:
new_trn_dset, new_tst_dset = UpdatedDataset.load_datasets(save_dir, metadata_type, x_prefix='old', y_prefix='new', z_prefix='new')

In [ ]:
comp_dset = CompareDataset(old_trn_dset, new_trn_dset)

In [ ]:
comp_dset.data(idx_type='max')

data_input_text : Mathematical finance
old lbl2data_input_text : ['Financial engineering', 'Outline of economics', 'List of economists', 'Financial economics', 'Quantitative behavioral finance', 'Mathematical finance', 'Technical analysis', 'Outline of finance', 'Derivative (finance)', 'Computational finance', 'Brownian model of financial markets', 'Economic model', 'Statistical finance', 'International Swaps and Derivatives Association', 'XVA', 'Index of accounting articles']
new lbl2data_input_text : ['Interest rate cap and floor#Black model', 'Credit valuation adjustment', 'Hull–White model', 'Greeks (finance)', 'Numerical analysis', 'Quantile function', 'Bond option#Valuation', 'Fourier transform', 'Longstaff–Schwartz model', 'Probability distribution', 'Expected value', 'Asymptotic analysis', 'Survival analysis', 'Intrinsic value (finance)', 'List of economists', 'Lattice model (finance)', 'Implied volatility', 'Stochastic differential equation', "Margrabe's formula", 'Black–Derma

In [ ]:
comp_dset.metadata('cat_meta', idx_type='max')

data_input_text : Cyperus compressus
old lbl2data_input_text : ['List of Cyperus species']
new lbl2data_input_text : ['List of Cyperus species']
old cat2data_input_text : ['Flora of Western Australia', 'Plants described in 1753', 'Cyperus']
new cat2data_input_text : ['Flora of Queensland', 'Flora of Togo', 'Flora of Sri Lanka', 'Flora of Taiwan', 'Flora of Illinois', 'Flora of Botswana', 'Flora of Jamaica', 'Flora of Maryland', 'Flora of Tibet', 'Flora of Liberia', 'Flora of Nicaragua', 'Flora of India', 'Flora of Burkina Faso', 'Flora of Laos', 'Flora of Vietnam', 'Flora of Florida', 'Flora of Angola', 'Flora of Peru', 'Flora of Seychelles', 'Flora of Kenya', 'Flora of Thailand', 'Flora of Western Australia', 'Flora of Egypt', 'Flora of Texas', 'Taxa named by Carl Linnaeus', 'Flora of Zambia', 'Flora of Pakistan', 'Flora of Cape Verde', 'Flora of Cambodia', 'Flora of the Philippines', 'Flora of Senegal', 'Flora of Sudan', 'Flora of Sierra Leone', 'Flora of Belize', 'Flora of Costa Ric